**NOTE: in this version of the notebook, the output is cleared, to make the file lighter.**

## Content <a id='content'></a>

[Introduction](#intro)

[Data overview, preprocessing, and EDA](#prep)

[ML](#ml)

[Summary and conclusions](#conclusions)

## Introduction <a id='intro'></a>

Sweet Lift Taxi company has collected historical data on taxi orders at airports. To attract more drivers during peak hours, we were asked to predict the amount of taxi orders for the next hour. Preliminary requirement: the RMSE metric on the test set should not be more than 48.

To implement the work we will:

- Load the data.
- Check that the data is free of issues — missing data, extreme values, and so on.
- Train different models with various hyperparameters (the test sample should be 10% of the initial dataset).
- Draw conclusions.

[Back to Content](#content)

## Data overview, preprocessing, and EDA <a id='prep'></a>

The data is stored in the `taxi.csv` file. The number of orders is in the `num_orders` column.

Let's load the data.

### Libraries

In [ ]:
import re

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

import math

from sklearn.model_selection import train_test_split, cross_val_score, TimeSeriesSplit
from sklearn.pipeline import Pipeline

from IPython.display import display
import plotly.express as px # advanced plotting
import plotly.graph_objects as go

import warnings
from time import time

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.stats.diagnostic import het_white, acorr_ljungbox

In [ ]:
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error, make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline

In [ ]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge

In [ ]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from scipy.stats import randint as sp_randint  # for initializing random integer values

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
import warnings
warnings.filterwarnings("ignore")

### File upload

As part of the uploading we will check memory usage and date types for a small subsample, to avoid any excessive use of the memory. We will use a load() function to avoid potential problems with the file pathes.

In [ ]:
# function `load` for a csv load with try-except and a number of rows limit
def load(filename, sep = ',', nrow = None, dtype = None, parse_dates = None):
    """
    i=In addition to the file name, the function takes nrows parameter
    for a particular number of rows to load. If None, then the file is loaded fully.
    """
    try:
        df_raw = pd.read_csv(filename, sep = sep, nrows = nrow, dtype = dtype, parse_dates = parse_dates)
    except:
        df_raw = pd.read_csv('/'+filename, sep = sep, nrows = nrow, dtype = dtype, parse_dates = parse_dates)
    return df_raw

In [ ]:
# check the dataset data types and memory usage on the first 500 rows
data_raw = load('datasets/taxi.csv', nrow = 500)

In [ ]:
data_raw.info(memory_usage = 'deep')

The column names are regular, the `datetime` column takes much of the memory as `object`, let's fix it.

In [ ]:
data_raw = load('datasets/taxi.csv', parse_dates = [0])

In [ ]:
data_raw.info(memory_usage = 'deep')

Great, we have ~26500 data points, no missing values, and the dataset uses 414 KB of the memory. Let's take a closer look at the data.

In [ ]:
data_raw.head()

In [ ]:
data_raw.tail()

Thus, we have a half year of 10-min data.

### Duplicates

Let's make sure that there are no explicit duplicates and we have one observation for each date.

In [ ]:
data_raw.duplicated().sum()

No explicit duplicates.

In [ ]:
len(data_raw[data_raw.duplicated('datetime')])

No duplicateds in the `datetime` column as well. Great, now, let's turn this dataframe into time series, explore it and perform some preparations.

### Time series EDA

In [ ]:
data = data_raw.set_index('datetime')
data.sort_index(inplace=True)

In [ ]:
data.head()

In [ ]:
moving_average_window = 6*24  # daily

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=data.index,
    y=data.num_orders,
    name='number of drives within 10-min slots',
    line=dict(color=px.colors.qualitative.Prism[5])
    )
)
fig.add_trace(go.Scatter(
    x=data.rolling(moving_average_window).mean().index,
    y=data.rolling(moving_average_window).mean().num_orders,
    name='moving average number of drives within',
    line=dict(color=px.colors.qualitative.Prism[1])
    )
)
fig.update_layout(
    legend_x=0.1, legend_y=0.95
)
fig.show()

Looks like the data exhibits some trend and, given the nature of the data, it will be reasonable to assume sone seasonality. In addition, it looks like the variablity of the data grows with the time. Let's apply the [`seasonal_decompose()` model](https://www.statsmodels.org/dev/generated/statsmodels.tsa.seasonal.seasonal_decompose.html) from the time series module of the [`statsmodels` library](https://www.statsmodels.org). This is a naive model using moving averages, it can be used in either in `additive` version or in `multiplicative`. We will apply the former (default), since we will assume that the variability in the taxi rides has an additive nature.

Note: `seadonal_decompose()` is a naive model; it first estimates the trend by applying a convolution filter to the data; after removing the trend from the series, the average of the de-trended series for each period is the returned seasonal component.

In [ ]:
data_decomposed = seasonal_decompose(data.resample('1D').sum())  # the daily average is returned as the seasonal component

In [ ]:
def decomposed_viz(df, titles, start_date, end_date):
    fig = make_subplots(rows=3,cols=1)

    fig.add_trace(go.Scatter(
        x=df.trend[start_date:end_date].index,
        y=df.trend[start_date:end_date],
        name=titles[1],
        line=dict(color=px.colors.qualitative.Prism[0])),
        row=1,
        col=1
        )

    fig.add_trace(go.Scatter(
        x=df.seasonal[start_date:end_date].index,
        y=df.seasonal[start_date:end_date],
        name=titles[2],
        line=dict(color=px.colors.qualitative.Prism[3])),
        row=2,
        col=1
        )

    fig.add_trace(go.Scatter(
        x=df.resid[start_date:end_date].index,
        y=df.resid[start_date:end_date],
        name=titles[3],
        line=dict(color=px.colors.qualitative.Prism[4]),
        mode="markers+lines"),
        row=3,
        col=1
        )

    fig.update_xaxes(row=2, col=1, matches='x')
    fig.update_xaxes(row=3, col=1, matches='x')

    fig.update_layout(
        legend_x=0.02, legend_y=0.98,
        title_text=titles[0]
    )
    fig.show()

In [ ]:
daily_titles = [
    'Seasonality in taxi ride data',
    'daily trend',
    'tedrended pattern',
    'daily residuals'
]
decomposed_viz(data_decomposed, daily_titles, '2018-03-01', '2018-08-31')

From this naive model, the trend part follows that the Sweet Lift Taxi company steadily grew its audience, from 1300 rides per day in the beginning of March, 2018, up to 3700 rides per day by the end of August, 2018.

The seasonality component demonstrates some interesting pattern:
- The average number of rides diminishes on weekends and Tuesdays;
- There is a peak on Mondays;
- The numbers grow from Tuesday to Thursday.

No seasonality, besides the weekly one, can be observed from this dataset, supposedly due to the limited timeframe.

We lack the information on the geographical scope of the rides in the dataset; however, the seasonality component observed suggests that it can be a relatively limited area, with specific traffic patterns.

Residuals, in general, are located around the zero line, although they seem to be autocorrelated; in addition, there is a period of about one month, from March 20 to April 20, when their variablity falls down.

As we mentioned ealier, we should explore what seems to be growing variability in the data. Let's add rolling standard deviation and the first difference to the plot:

In [ ]:
data_daily = data.resample('1d').sum()

In [ ]:
data_daily['daily_difference'] = data_daily-data_daily.shift()

In [ ]:
data_daily.head()

In [ ]:
num_of_sub_plots = 5
fig = make_subplots(rows=num_of_sub_plots,cols=1, )
for i in range(1,num_of_sub_plots):
    fig.update_xaxes(row=i+1, col=1, matches='x')


fig.add_trace(go.Scatter(
    x=data_decomposed.trend.index,
    y=data_decomposed.trend,
    name='daily trend',
    line=dict(color=px.colors.qualitative.Prism[0]),
    legendgroup = '1'),
    row=1,
    col=1,
    )

fig.add_trace(go.Scatter(
    x=data_decomposed.seasonal.index,
    y=data_decomposed.seasonal,
    name='daily trend',
    line=dict(color=px.colors.qualitative.Prism[3]),
    legendgroup = '2'),
    row=2,
    col=1,
    )

fig.add_trace(go.Scatter(
    x=data_decomposed.resid.index,
    y=data_decomposed.resid,
    name='daily residuals',
    line=dict(color=px.colors.qualitative.Prism[4]),
    legendgroup = '3',
    mode="markers+lines"),
    row=3,
    col=1,
    )

fig.add_trace(go.Bar(
    x=data.iloc[:,0].resample('1d').std().index,
    y=data.iloc[:,0].resample('1d').std(),
    name='',
    marker=dict(color=px.colors.qualitative.Prism[6]),
   legendgroup = '4'),
    row=4,
    col=1,
    )

fig.add_trace(go.Scatter(
    x=data.iloc[:,0].resample('1d').std().index,
    y=data.iloc[:,0].resample('1d').std(),
    name='rolling standard deviation',
    mode="markers",
    marker=dict(color=px.colors.qualitative.Prism[6]),
    legendgroup = '4'),
    row=4,
    col=1,
    )

fig.add_trace(go.Scatter(
    x=data_daily.index,
    y=data_daily.daily_difference,
    name='daily difference in number of orders',
    line=dict(color=px.colors.qualitative.Prism[7]),
    legendgroup = '5',
    mode="markers+lines"),
    row=5,
    col=1,
    )

fig.update_layout(
    legend_x=0.95, legend_y=0.99,
    title_text="Daily trend and seasonality in taxi ride data",
    height=850, width=980,
    legend_bgcolor='rgba(0,0,0,0)',
    legend_tracegroupgap = 120,
)
fig.show()

The standard deviation grows with the growing number of rides; it exhibits weekly spikes as well - they start to be more expressed by the end of the period under investigation.

#### Data upload summary

1. As part of the uploading process we checked memory usage and date types for a small subsample, which allowed us to properly upload the data.
2. The raw dataset has ~26500 10-min observation points, no missing values, no duplicates.
3. The data exhibits a trend: the overall number of rides per day grew from 1300 in the beginning of March, 2018, up to 3700 rides per day by the end of August, 2018.
4. The seasonality component demonstrates some interesting pattern:
- The average number of rides diminishes on weekends and Tuesdays;
- There is a peak on Mondays;
- The numbers grow from Tuesday to Thursday.
5. The standard deviation grows with the growing number of rides; it exhibits weekly spikes as well - they start to be more expressed by the end of the period under investigation.

From these results, the need appears to conduct heteroscedasticity and autocorrelation tests to check our assumtions, before we dive into modeling. Let's illustrate the required testing methods on `statsmodels` naive decomposition model.

### Testing for Heteroscedasticity for Naive Decomposition

Heteroscedasticity means that conditional variance, i.e. the variability of the dependent (target) variable `y` for each value of the explanatory variables, `t` for time series, in the data is not constant. One of heteroscedasticity forms is when the amount of fluctuation depends on the value of the target variable. It can be introduced by the measurement process or be the intrinsic characteristic of the process under investigation. Heteroscedasticity makes any model, which assumes that the conditional variance of the independent variable is constant, i.e. the residual error for each prediction is identically (and, in many cases, normally) distributed and, thus,  has the same probability distribution, unreliable: standard errors of the model's parameters can become incorrect and misleading when inference should be made with regard to the significance level of the model parameters.

There is a family of tests for heteroscedasticity; all of them share the same approach, though employ different functions for modeling the dependence of fluctuations in the data on the target variable (you can find [intro](https://towardsdatascience.com/heteroscedasticity-is-nothing-to-be-afraid-of-730dd3f7ca1f) here). We will run the most popular, White's, test on the residuals of the decomposed initial time series.

Let's formulate the hypothesis:
1. Null Hypothesis: the residuals are distributed with equal variance (homoscedasticity is present)
2. Alternative Hypothesis: the residuals are not distributed with equal variance (heteroscedasticity is present)
3. We will use 0.05 as the signifcance level parameter (alpha); if the p-value of the test results is smaller than alpha then we can reject H0 and conclude that the data is heteroscedastic. 

Note 1: the test output contains two alternatives:
- original test statistic
- statistic of the hypothesis that the error variance does not depend on the explanatory variables `X`, `t` and a constant in our case (f-statistics).

Note 2: in this we are using the residuals of the naive decomposition of the daily aggregate data from the previous section; it will be sufficient in case H1 is accepted.

In [ ]:
#perform White's test
white_test = het_white(
    np.array(data_decomposed.resid.dropna()),                             # NaN values should be dropped
    sm.add_constant(np.array(range(len(data_decomposed.resid.dropna())))) # simplify the time domain into an integer value
                                                                          # and add constant
)
#define labels to use for output of White's test
labels = ['Test Statistic', 'Test Statistic p-value', 'F-Statistic', 'F-Test p-value']
#print results of White's test
for key in dict(zip(labels, white_test)):
    print(f'{key}: {dict(zip(labels, white_test))[key]}')

Note: `data_decomposed.resid` contains NaN values at the beginning and at the end of the series, due to the calculation method, they should be dropped:

In [ ]:
data_decomposed.resid[data_decomposed.resid.isna()]

Both p-values of the test results are smaller than alpha: we can reject H0 and conclude that the data is heteroscedastic, which should be accounted for when modeling the process. There are two ways to tackle the heteroscedasticity of the data: use a heteroscedasticity-robust model, or transform (scale) the initial data; such transformation should turn:
- the variance into "more" constant, i.e. lower the heteroscedasticity in the data;
- the residual errors of the model into normally or "almost" normally distributed.
- the relationship between the explanatory variables and the dependent variable into additive.

Let's proceed to autocorrelation.

### Testing for Autocorrelation for Naive Decomposition

Let’s plot the data against a time lagged version of itself for lags from 1 to 30 [days].

In [ ]:
data.head()

In [ ]:
data_lagged = data.copy()
lag_num_week = 6*24*7 # x6 - hour, x24 - day, x7 - week
names = ['num_orders'] + ['lag_'+str(i) for i in range(1, lag_num_week+1)]
data_lagged = pd.concat([data_lagged['num_orders'].shift(i) for i in range(0, lag_num_week+1)],
                        axis=1)
data_lagged.columns = names

In [ ]:
corr=data_lagged.corr()

In [ ]:
px.imshow(
        corr,
        x=corr.columns.to_list(),
        y=corr.columns.to_list(),
        color_continuous_scale='viridis', zmin=-0, zmax=1,
        )

We can observe the following correlations within the week time period:
- daily correlation close to 1;
- autocorrelation of closest lags, `t-1` to `t-7` can be characterized as high, while it diminishes down to ~0.4;
- although in a very moderate way, correlations can be tracked within the day also: the data starts at 12 am, so the pattern shows spikes of activity around 08:00 (`t-48`) am to 10:00 am, 02:00 (`t-84`) pm t 04:00 pm, and at 07:00 (`t-114`) pm.
- additional correlation can be observed at night, between 10:00 pm to and 03:00 (`t-132` to `t-144`) and (`t-1` to `t-18`) am.

Let's make a close up into "a day in Sweet Lift Taxi's life".

In [ ]:
data_lagged_short = data.copy()
lag_num_day = 6*24 # x6 - hour, x24 - day
names = ['num_orders'] + ['lag_'+str(i) for i in range(1, lag_num_day+1)]
data_lagged_short = pd.concat([data_lagged_short['num_orders'].shift(i) for i in range(0, lag_num_day+1)],
                        axis=1)
data_lagged_short.columns = names

In [ ]:
corr_short = data_lagged_short.corr()

In [ ]:
px.imshow(
        corr_short,
        x=corr_short.columns.to_list(),
        y=corr_short.columns.to_list(),
        color_continuous_scale='viridis', zmin=-0, zmax=1,
        )

Let's test for the autocorrelation. Two statistical tests - Ljung-Box and Durbin-Watson - are used roughly for to check the autocorrelation in a data series; however, Ljung-Box can be used for any lag value, while Durbin-Watson can be used just for the lag of 1.

Let's formulate the hypothesis:
1. Null Hypothesis:  the residuals are independently distributed.
2. Alternative Hypothesis: the residuals exhibit serial correlation.
3. We will use 0.05 as the signifcance level parameter (alpha); if the p-value of the test results is higher than alpha then we fail to reject H0 and conclude that the lagged time series are independently distributed. 

Note 1: in this section we are testing autocorrelation within intra-day data; thus, we should modify our decomposition to account for trend and seasonality, while applying the test to residuals of the required frequency. Let's check decomposition with the original frequency and for hourly aggregated data.

Note 2: To preserve the daily trend calculation, we will employ the `period` parameter of the `seasonal_decompose` model to override the default frequency of the timeseries index passed to the model.

In [ ]:
data_hf_decomposed = seasonal_decompose(data, period=lag_num_week)

Now, if we soom in, we can observe the trend and the "seasonality" on the daily level:

In [ ]:
start_day = '2018-05-06'
end_day = '2018-05-20'

In [ ]:
titles = [
    'Daily patterns in taxi ride data',
    'daily trend',
    'intra-day pattern',
    'intra-day residuals'
]
decomposed_viz(data_hf_decomposed, titles, start_day, end_day)

Now, let's see how hourly aggregation looks like:

In [ ]:
data_hourly_decomposed = seasonal_decompose(data.resample('60min').sum(), period=lag_num_week)

In [ ]:
titles = [
    'Daily patterns in taxi ride data',
    'daily trend',
    'hourly pattern',
    'hourly residuals'
]
decomposed_viz(data_hourly_decomposed, titles, start_day, end_day)

For testing purposes, we will use the daily model.

Note 2: the residual series has NaNs for the first and the last three weeks; thus, for testing purposes we will omit these observations:

In [ ]:
data_hourly_decomposed.resid[500:505]

In [ ]:
lj_test_hourly_df = data_hourly_decomposed.resid.dropna()

In [ ]:
lj_test_hourly_df.tail()

In [ ]:
lj_res_hourly_df = sm.stats.acorr_ljungbox(lj_test_hourly_df,
                                24,  # number of lags to test
                                return_df=True) # as mentioned above, NaNs must be dropped 

In [ ]:
lj_res_hourly_df.head()

Let's check how many Ljung-Box p-values are higher than alpha:

In [ ]:
lj_res_hourly_df[lj_res_hourly_df.lb_pvalue>0.05]

For this basic decomposition model, Ljung-Box test shows that all the lags (at least within one day) are highly correlated. Let's apply the `plot_acf` function of the `statsmodels` library to visually check the things:

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf
plot_acf(lj_test_hourly_df, lags = 24)
plt.show()

This method shows that only part of the lags have statistically significant autocorrelation, namely `t-1`, `t-2`, `t-7`, `t-8`, `t-11`, `t-12`, `t-13`, `t-22`, `t-23`, `t-24`, which we already mentioned before. We should remember though that ACF only looks at particular lags and tests randomness of each tag, while  Ljung-Box test tests whether a group of autocorrelations of a time series is random.

### EDA summary

At the data exploration step, we showed that for further modeling we should be aware of the following characteristics of the initial time series:
- heteroscedasticity;
- autocorrelative nature;
- overall nonlinear dependency on time.

[Back to Content](#content)

## Modeling the intra-day demand for taxi rides <a id='ml'></a>

### Feature engineering

We were asked to model hourly data; thus, we will resample the source data and, based on the EDA conclusions, add the following features:
- 24 hourly lags,
- day of week, week, and hour,
- 4-hour slot rolling median (to make the model more robust).

In [ ]:
model_data = data.resample('60min').sum().copy()
lag_num = 24
names = ['num_orders'] + ['lag_'+str(i) for i in range(1, lag_num+1)]
model_data = pd.concat([model_data['num_orders'].shift(i) for i in range(0, lag_num+1)],
                        axis=1)
model_data.columns = names

In [ ]:
model_data['dayofweek'] = model_data.index.dayofweek
model_data['week'] = model_data.index.isocalendar().week
model_data['hour'] = model_data.index.hour

Rolling median should be calculated on shifted data ro avoid target leakage:

In [ ]:
width = 4

In [ ]:
model_data['rolling_median'] = model_data['num_orders'].shift(width-1).rolling(width).median()

In [ ]:
model_data.dropna(inplace=True)

### Scorer

We were asked to use RMSE and achieve the metric value of not more than 48 on the test sample.

In [ ]:
def rmse(y_true: pd.Series, y_pred:pd.Series) -> float:  
    """Calculate the RSME metric"""
    rmse_score = mean_squared_error(y_true, y_pred) ** 0.5 
    return rmse_score

We will [`make_scorer()`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html) function to wrap `rmse` scoring function for use in `cros_val_score`. In our case, the lower is the metric value the better; thus, we should define the `greater_is_better` parameter of the `make_scorer()` function to be `False`. In this case, the scorer object will sign-flip the outcome, which should be taken into account.

In [ ]:
rmse_score = make_scorer(rmse, greater_is_better=False)  # rmse as a user-defined scoring function

### Splitting the data

In [ ]:
X = model_data.drop('num_orders', axis = 1)
Y = model_data['num_orders']

Time series must be split without shuffling:

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.1, shuffle=False, random_state = 54321)

### A note on cross-validation

Despite the fact that we are dealing with a time series, we still want to take advantage of the cross-validation technique is a well-established methodology for hyper-parameter tuning and feature choosing. For time series, the dependence on previous observations can cause data leakage from response variables to lag variables. Thus, in this case, we should choose a way for performing cross-validation which would be adapted to solve issues we have encountered in the EDA section.

In a general case of cross-validation, the training set is further split into k folds. During each iteration of the cross-validation, one fold is held as a validation set and the remaining folds are used for training. This allows us to make the best use of the data and avoid biasing the model towards patterns that may be overly represented in a given fold. Then the score obtained on all folds is averaged and the standard deviation is calculated. For the time series, the training set should be devided into two folds at each iteration, while the validation set is always ahead of the training split.

One of the methods in [`TimeSeriesSplit`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.TimeSeriesSplit.html), which provides train/validate indices to split time series data samples that are observed at fixed time intervals, in train/validate sets. In each split, validate indices are higher than before, and thus shuffling in cross validator is inappropriate: in the kth split, it returns first k folds as train set and the (k+1)th fold as test set. However, this method of time series splitting can result in data leakage, while the model will look at future observations, which are supposed to be used as lags for validation purposes. To tackle this issue, blocked cross-validation was introduced.

Blocked cross-validation works by adding margins abetween the training and validation folds in order to prevent the model from observing lag values which are used twice once as a regressor and another as a response. The blocked splitter should be [introduced explicitely](https://medium.com/sci-net/cross-validation-strategies-for-time-series-forecasting-9e6cfab91f60):

In [ ]:
class BlockingTimeSeriesSplit():
    def __init__(self, n_splits):
        self.n_splits = n_splits
    
    def get_n_splits(self, X, y, groups):
        return self.n_splits
    
    def split(self, X, y=None, groups=None):
        n_samples = len(X)
        k_fold_size = n_samples // self.n_splits
        indices = np.arange(n_samples)

        margin = 24  # we are taking margin of 24 hours between the training fold and the validation fold
        for i in range(self.n_splits):
            start = i * k_fold_size
            stop = start + k_fold_size
            mid = int(0.8 * (stop - start)) + start
            yield indices[start: mid], indices[mid + margin: stop]

In [ ]:
btscv = BlockingTimeSeriesSplit(n_splits=5)

[Back to Content](#content)

### Baseline Regression

In [ ]:
from sklearn import linear_model

In [ ]:
baseline_check = linear_model.LinearRegression()

In [ ]:
start = time()
res = cross_val_score(baseline_check, X_train[['lag_1', 'hour']], Y_train, cv=btscv, scoring=rmse_score).mean()
stop = time()
print(f'The mean across 5 evaluation scores equals: {-res:.0f}.')
modeling_time = stop - start
print(f'Modeling time: {modeling_time}.')

As a baseline, we run a simple Regression with only two features, `hour` abd `lag_1`, the RMSE score of this model ended up to be 32.

### Random Forest

To make the search for hyperparameters more feasible, we will empoloy the Random Search technique, namely the `RandomizedSearchCV` method. `RandomizedSearchCV` is relatively cost- (computationally less intensive) and time-effective (faster – less computational time), e.g. as compared to the `GridSearchCV` method: [link](https://jmlr.csail.mit.edu/papers/volume13/bergstra12a/bergstra12a.pdf). To perform the cross-validation we will use the `cv` parameter.

In [ ]:
pipe = Pipeline([('rf', RandomForestRegressor(random_state=12345))])

In [ ]:
rs_rf = RandomizedSearchCV(
    pipe,
    param_distributions = {"rf__max_depth": range(2,10),
              "rf__min_samples_split": sp_randint(2, 10),
              "rf__min_samples_leaf": sp_randint(2, 30),
              "rf__max_leaf_nodes": sp_randint(2,40),
              "rf__n_estimators": sp_randint(20,60)
             },
    scoring=rmse_score,
    n_iter=50,
    cv = btscv
)

In [ ]:
start = time()
rs_rf.fit(X_train, Y_train)
stop = time()

In [ ]:
search_time = stop - start
print(f'Search time: {search_time}.')

In [ ]:
rs_rf.best_params_

In [ ]:
- rs_rf.best_score_  # remember that the scorer object sign-flips the outcome, since the lower score the better

Thus, the best model parameters:

In [ ]:
best_rf_params = {}
for key in rs_rf.best_params_:
    best_rf_params[key[4:]] = rs_rf.best_params_[key]
best_rf_params

In [ ]:
best_rf_model_1 = RandomForestRegressor(random_state = 12345, **best_rf_params)

In [ ]:
pipe_best_rf_1 = Pipeline([('model', best_rf_model_1)])

Let's build the predicted time series to illustrate the result:

In [ ]:
start = time()
pipe_best_rf_1.fit(X_train, Y_train)
stop = time()
best_rf_fit_time = stop - start
print(f'Best RandomForest model fit time: {best_rf_fit_time}.')
pipe_best_rf_predictions = pipe_best_rf_1.predict(X_train)

In [ ]:
print(f"The train dataset RMSE score for the best RandomForest model: {rmse(Y_train, pipe_best_rf_predictions):.0f}.")

The final RMSE metric value for the train dataset is 22, let's visualize the train and the predicted arrows:

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=Y.index,
    y=Y,
    name='actual number of drives in the dataset',
    line=dict(color=px.colors.qualitative.Prism[3])
    )
)
fig.add_trace(go.Scatter(
    x=Y_train.index,
    y=pipe_best_rf_predictions,
    name='predicted number of drives for the train dataset',
    line=dict(color=px.colors.qualitative.Prism[5])
    )
)

fig.update_layout(
    legend_x=0.1, legend_y=0.95
)
fig.show()

We can see, that the model actually performs decently on the train data; however, while catching the overall trend and even seasonality, still misses on the variance spikes. Let's check out the residuals.

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=Y_train,
    y=Y_train-pipe_best_rf_predictions,
    name='residual vs response',
    mode='markers',
    line=dict(color=px.colors.qualitative.Prism[4])
    )
)
fig.update_xaxes(title_text='actual number of drives in the test dataset')
fig.update_yaxes(title_text='residuals of the Random Forest model')
fig.update_layout(legend_x=0.1, legend_y=0.95, title = 'Residual plot')
fig.show()

There is obvious linear dependency of the residuals on the response variable; this is due to untreated heteroscedasticity.

To tackle the problem of heteroscedasticity, we can apply one of the following methods:
- transforming the target variable (the most common way is taking a log);
- redefine the target variable, e.g. by taking difference as a new dependent variable;
- using regression models which account for heteroscedasticity, like weighted regression (gives smaller weights to data points that have higher variances, which shrinks their squared residuals) or (G)ARCH (Autoregressive Conditional Heteroskedasticity) models.

Let's check whether we can transform our target variable by tacking log:

In [ ]:
np.log(model_data.num_orders).describe()

Unfortunately, on this path we encounter a problem: the infinite negative values. Looks like we have zero value in the target variable:

In [ ]:
model_data[model_data.num_orders==0]

Right, there is one zero value. Let's check whether taking difference can tackle the problem:

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=model_data.index,
    y=(model_data.num_orders - model_data.lag_1),
    line=dict(color=px.colors.qualitative.Prism[8])
    )
)
fig.update_yaxes(title_text='delta')
fig.update_layout(legend_x=0.1, legend_y=0.95, title = 'Taking difference of reposnse variable')
fig.show()

This option does not work either: the growing variability keeps appearing. Let's check GARCH.

[Back to Content](#content)

### Intro to GARCH

ARCH models explicitly model the change in variance over time in a time series,  by incorporating lag squared residual errors into the model. ARCH assumes the series to be stationary, other than the change in variance, meaning it does not have a trend or seasonal component. Generalized Autoregressive Conditional Heteroskedasticity, or GARCH, is an extension of the ARCH model that incorporates a moving average component together with the autoregressive components of variance by incorporating lag variance terms and lag residual errors from a mean process.

As such, the model introduces two parameters, `p` and `q`, that describe:
- `p`: The number of lag variances to include in the GARCH model.
- `q`: The number of lag residual errors to include in the GARCH model.

A generally accepted notation for a GARCH model is to specify the GARCH() function with the p and q parameters GARCH(p, q); a GARCH model subsumes ARCH models, where a GARCH(0, q) is equivalent to an ARCH(q) model.

What about the autoregressive components and moving average components of the mean? Based on the EDA results, we have to account for them as well. To this end, we have to apply ARIMA modeling.

### Intro to ARIMA

ARIMA (Integrated Autoregression and Moving Average) models use a linear combination of past observations and residuals to account for trend and seasonal components. The general form, [SARIMA (Seasonal Autoregressive Integrated Moving Average)](https://otexts.com/fpp2/seasonal-arima.html#), specifies the autoregression (AR), differencing (I), and moving average (MA) hyperparameters for the seasonal component of the series, as well as an additional parameter for the period of the seasonality; thus, configuring a [SARIMAX(p, d, q)x(P, D, Q, s)](https://www.statsmodels.org/stable/generated/statsmodels.tsa.arima.model.ARIMA.html) allows selecting hyperparameters for both the trend and seasonal elements of the series:
- trend parameters
    - `p`: Trend autoregression order,
    - `d`: Trend difference order,
    - `q`: Trend moving average order,
- seasonal parameters
    - P: Seasonal autoregressive order,
    - D: Seasonal difference order,
    - Q: Seasonal moving average order,
    - s: The number of time steps for a single seasonal period, e.g. with monthly data (and `s = 12`),
        - a seasonal first order autoregressive model would use x(t-12) to predict x(t).
        - a seasonal second order autoregressive model would use x(t-12) and x(t-24) to predict x(t).
    
The SARIMA model can subsume the ARIMA, ARMA, AR, and MA models via model configuration parameters:
- autoregressive models: AR(p)
- moving average models: MA(q)
- mixed autoregressive moving average models: ARMA(p, q)
- integration models: ARIMA(p, d, q)
- seasonal models: SARIMA(P, D, Q, s)

Additionaly, we should relate to the `t` parameter, which is responsible for the trend: can be specified as a string where `'c'` indicates a constant term, `'t'` indicates a linear trend in time, and `'ct'` includes both. Can also be specified as an iterable defining a polynomial, as in [`numpy.poly1d`](https://numpy.org/doc/stable/reference/generated/numpy.poly1d.html).


Our previous analysis suggested some ideas for these hyperparameter, but precise configuring of the model seems to take more time and deeper analysis. Alternatively, we can grid search the best configuration among a predefined set of possibilities.

### ARIMA + GARCH

So, GARCH tackles the problem of correct estimate of conditional variance, i.e. deals with residuals, while ARIMA estimates the value of the target variable itseld. That means that we can combine ARIMA and GARCH to model both non-stationary and heteroscedastic time series. One way to implement this, despite the [comments](https://stats.stackexchange.com/questions/77925/procedure-for-fitting-an-arma-garch-model) of not being [consistent](https://en.wikipedia.org/wiki/Consistency_(statistics)), is to fit the GARCH model on the residuals of the ARIMA model instead of the target variable itself. The second option, less straightforward, but consistent, is to create simultaneous estimation.

Let's illustrate the ARIMA modeling:

In [ ]:
arima_rmse_values = []

In [ ]:
for train_index, test_index in btscv.split(Y_train):  # alternatively, tscv = TimeSeriesSplit(n_splits = ...) can be used
    cv_train, cv_test = Y_train.iloc[train_index], Y_train.iloc[test_index]
    arima =  sm.tsa.arima.ARIMA(cv_train, order=(24,0,4)).fit(method='innovations_mle')
    
    predictions = arima.predict(cv_test.index[0], cv_test.index[-1])
    true_values = cv_test.values
    arima_rmse_values.append((mean_squared_error(true_values, predictions))** 0.5 )
    
print("RMSE: {}".format(np.mean(arima_rmse_values)))

The `.summary()` method prints out the results:

In [ ]:
arima.summary()

Note: initially, `s=168` was included as well, but these models are the slowest and the final score for them is lower, so we will run with maximum seasonal parameter value fo 24.

In [ ]:
def sarima_configs(seasonal=24*7):
    models = list()
    # define config lists
    p_params = [2, 4]
    d_params = [0, 1, 2]
    q_params = [1, 2, 4]
    P_params = [0, 1, 2]
    D_params = [0, 1]
    Q_params = [0, 2]
    s_params = [24]
    # create config instances
    for p in p_params:
        for d in d_params:
            for q in q_params:
                for P in P_params:
                    for D in D_params:
                        for Q in Q_params:
                            for s in s_params:
                                cfg = [(p,d,q), (P,D,Q,s)]
                                models.append(cfg)
    return models

In [ ]:
cfg_list = sarima_configs()

In [ ]:
len(cfg_list)

We will run 216 combinations of parameters and choose the one with the lower score. Let's define a function to run the model in a loop:

In [ ]:
def arima_run(time_series, param_cfg):
    rmse_values =[]
    for train_index, test_index in btscv.split(time_series):
        try:
            cv_train, cv_test = time_series.iloc[train_index], time_series.iloc[test_index]
            arima =  sm.tsa.arima.ARIMA(
                cv_train,
                order=param_cfg[0],
                seasonal_order=param_cfg[1],
                trend='c'
            ).fit(method='innovations_mle')

            predictions = arima.predict(cv_test.index[0], cv_test.index[-1])
            true_values = cv_test.values
            rmse_values.append((mean_squared_error(true_values, predictions))** 0.5 )
        except:
            return None
    
    return np.mean(rmse_values)

In [ ]:
arima_gs_results = {}

We will iterate over the configuration list, but should take into account that not the all models will converge or converge fast (**NOTE: in this version of the notebook, the output was cleared, since the file ended up to be too large to be uploaded; be cautious that the cell below runs for long time; for more convenience, the `arima_best_params` value is passed manualy 5 cells below**).

In [ ]:
for i in range(0, len(cfg_list)):
    result = None
    print(f'Current configuration: {cfg_list[i]}')
    try:
        result = arima_run(Y_train, cfg_list[i])
    except:
        error = None
    if result is not None:
        print(f'Mean RMSE: {result}')
    else:
        print(f'Model did not converge.')
    arima_gs_results[i] = {'configuration': cfg_list[i], 'rmse': result}

In [ ]:
# arima_gs_results

In [ ]:
arima_gs_results_list = [x[1] for x in arima_gs_results.items() if x[1]['rmse'] is not None]

In [ ]:
sorted(arima_gs_results_list, key=lambda x: x['rmse'])

Neither of the models showed a lower score than the Random Forest model above. Due to complexity of calculations, we will choose the second best one which consumes the least time.

**See the NOTE above.**

In [ ]:
best_result = sorted(arima_gs_results_list, key=lambda x: x['rmse'])[1]
best_result

In [ ]:
arima_best_params = [(2, 0, 1), (2, 0, 2, 24)]

In [ ]:
arima_best =  sm.tsa.arima.ARIMA(
    Y_train,
    order=arima_best_params[0],
    seasonal_order=arima_best_params[1],
    trend='c'
).fit(method='innovations_mle')

In [ ]:
predictions = arima_best.predict(Y_train.index[0], Y_train.index[-1])

In [ ]:
arima_residuals = Y_train.values - predictions

In [ ]:
# pip install arch

In [ ]:
from arch.univariate import ZeroMean, GARCH

In [ ]:
garch_model = ZeroMean(arima_residuals)  # applied when residuals from a model estimated separately

Note: the entire data is passed to the model in when initializing it; the model is trained up to a split data and forecasts are only produced for observations after the final observation used to estimate the model [link](https://arch.readthedocs.io/en/latest/univariate/forecasting.html).

In [ ]:
split_date = '2018-07-13 01:00:00'

In [ ]:
garch_model.volatility = GARCH(p=3, q=3
                              )  # checked several combinations
res = garch_model.fit(update_freq=0, disp="off", last_obs=split_date)
print(res.summary())

In [ ]:
res.plot()
plt.show()

For GARCH(3,3) model, all beta coefficients are significant, let's visualize the GARCH residual diagnostics [source code](https://medium.com/tej-api-financial-data-anlaysis/data-analysis-10-arima-garch-model-part-1-a011bf45f66c):

In [ ]:
import statsmodels.api as sm
import statsmodels.graphics.tsaplots as sgt

garch_std_resid = pd.Series(res.resid / res.conditional_volatility)
fig = plt.figure(figsize = (15, 8))

# Residual
garch_std_resid.plot(ax = fig.add_subplot(3,1,1), title = 'GARCH Standardized-Residual', legend = False)

# ACF/PACF
sgt.plot_acf(garch_std_resid, zero = False, lags = 40, ax=fig.add_subplot(3,2,3))
sgt.plot_pacf(garch_std_resid, zero = False, lags = 40, ax=fig.add_subplot(3,2,4))

# QQ-Plot & Norm-Dist
sm.qqplot(garch_std_resid, line='s', ax=fig.add_subplot(3,2,5)) 
plt.title("QQ Plot")
fig.add_subplot(3,2,6).hist(garch_std_resid, bins = 40)
plt.title("Histogram")

plt.tight_layout()
plt.show()

The residual diagnostics shows that they have heavy right tail, which means they *still* do not come from a normal distribution ([more on GARCH residual diagnostics](https://data.library.virginia.edu/understanding-q-q-plots/)).

### ARIMA-GARCH - conclusions

We have run Grid Search for 200+ sets of the ARIMA model and found the best combination, in terms of time consumption and RMSE score. Neither of the ARIMA models showed higher RMSE value than the cross-validated RMSE of the initial Random Forest model, but our goal was to check whether we can improve the state by applying a GARCH model to ARIMA residuals. The study showed that the lag variance and lag residual error components can be explanatory variables of statistical significance; however, the GARCH model failed to produced normally distributed residuals as well.

Despite being ambiguous and time-consuming, the results of diving into ARIMA-GARCH modeling suggest that re-working the initial Random Forest by including additional features, namely rolling standard deviation (a proxy to variance) and a number of its lags, can benefit the model. Below, we check this idea.

[Back to Content](#content)

### Re-working the Random Forest model

We should start from the very beginning and add the features mentioned above. We want to avoid splitting the data again, so we will use indices to construct 'augmented' `X_train` and `X_test` datasets and their target values.

In [ ]:
model_data_extra = data.resample('60min').sum().copy()
model_data_extra = pd.concat([model_data_extra['num_orders'].shift(i) for i in range(0, lag_num+1)],
                        axis=1)
model_data_extra.columns = names
model_data_extra['dayofweek'] = model_data_extra.index.dayofweek
model_data_extra['week'] = model_data_extra.index.isocalendar().week
model_data_extra['hour'] = model_data_extra.index.hour
width = 4
model_data_extra['rolling_median'] = model_data['num_orders'].shift(width-1).rolling(width).median()
model_data_extra['rolling_std'] = model_data_extra['num_orders'].shift(width-1).rolling(width).std()
model_data_extra['rolling_std_lag_1'] = model_data_extra['rolling_std'].shift(1)
model_data_extra['rolling_std_lag_2'] = model_data_extra['rolling_std'].shift(2)
model_data_extra['rolling_std_lag_3'] = model_data_extra['rolling_std'].shift(3)

In [ ]:
X_extra = model_data_extra.drop('num_orders', axis = 1)

In [ ]:
X_train_indices = X_train.index
X_train_extra = X_extra.loc[X_train_indices,:] 

In [ ]:
X_test_indices = X_test.index
X_test_extra = X_extra.loc[X_test_indices,:] 

We should drop additional rows with NaN values which showed up due to shifting and adjust the target variable subsets:

In [ ]:
X_train_extra.dropna(inplace=True)
X_test_extra.dropna(inplace=True)

In [ ]:
Y_train_extra = Y_train.loc[X_train_extra.index]
Y_test_extra = Y_test.loc[X_test_extra.index]

Now we have the updated `X_train` and `X_test` datasets, and can re-run the model.

In [ ]:
pipe_extra = Pipeline([('rf', RandomForestRegressor(random_state=12345))])

In [ ]:
rs_rf_extra = RandomizedSearchCV(
    pipe_extra,
    param_distributions = {"rf__max_depth": range(2,10),
              "rf__min_samples_split": sp_randint(2, 10),
              "rf__min_samples_leaf": sp_randint(2, 30),
              "rf__max_leaf_nodes": sp_randint(2,40),
              "rf__n_estimators": sp_randint(20,60)
             },
    scoring=rmse_score,
    n_iter=100,  # add number of iterations to make a more dense search
    cv = btscv
)

In [ ]:
start = time()
rs_rf_extra.fit(X_train_extra, Y_train_extra)
stop = time()

In [ ]:
search_time_extra = stop - start
print(f'Search time: {search_time_extra}.')

In [ ]:
rs_rf_extra.best_params_

In [ ]:
- rs_rf_extra.best_score_

Thus, the best model parameters:

In [ ]:
best_rf_extra_params = {}
for key in rs_rf_extra.best_params_:
    best_rf_extra_params[key[4:]] = rs_rf_extra.best_params_[key]
best_rf_extra_params

In [ ]:
best_rf_model_2 = RandomForestRegressor(random_state = 12345, **best_rf_extra_params)

In [ ]:
pipe_best_rf_2 = Pipeline([('model', best_rf_model_2)])

Let's build the predicted time series to illustrate the result:

In [ ]:
start = time()
pipe_best_rf_2.fit(X_train_extra, Y_train_extra)
stop = time()
best_rf_fit_time = stop - start
print(f'Best RandomForest model fit time: {best_rf_fit_time}.')
pipe_best_rf_predictions = pipe_best_rf_2.predict(X_train_extra)

In [ ]:
print(f"The train datset RMSE score for the best RandomForest model: {rmse(Y_train_extra, pipe_best_rf_predictions):.0f}.")

The final RMSE metric value for the train dataset is 22, the same as in the first Random Forest model.

OK, let's apply it to the test data subset and visualize:

In [ ]:
best_rf_extra_params = {}
for key in rs_rf_extra.best_params_:
    best_rf_extra_params[key[4:]] = rs_rf_extra.best_params_[key]
final_rf_extra_model = RandomForestRegressor(random_state = 12345, **best_rf_extra_params)
pipe_best_rf_extra = Pipeline([('model', final_rf_extra_model)])
start = time()
pipe_best_rf_extra.fit(X_train_extra, Y_train_extra)
stop = time()
best_rf_extra_fit_time = stop - start
print(f'Best RandomForest model fit time: {best_rf_extra_fit_time}.')
pipe_best_rf_extra_predictions = pipe_best_rf_extra.predict(X_test_extra)
print(f"The test datset RMSE score for the best RandomForest model: {rmse(Y_test_extra, pipe_best_rf_extra_predictions):.0f}.")

For the modified set of features, the final RMSE metric value for the test dataset is 44, which is better than the requested threshold of 48, altough the difference between the cross-validation score is pretty drastic, which can be indication for overfitting. Let's visualize the test and the predicted arrows:

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=Y.index,
    y=Y,
    name='actual number of drives in the test dataset',
    line=dict(color=px.colors.qualitative.Prism[3])
    )
)
fig.add_trace(go.Scatter(
    x=X_train_extra.index,
    y=final_rf_extra_model.predict(X_train_extra),
    name='predicted number of drives for the train dataset',
    line=dict(color=px.colors.qualitative.Prism[5])
    )
)
fig.add_trace(go.Scatter(
    x=X_test_extra.index,
    y=pipe_best_rf_extra_predictions,
    name='predicted number of drives for the test dataset',
    line=dict(color=px.colors.qualitative.Prism[5], dash='dot')
    )
)
fig.update_layout(
    legend_x=0.1, legend_y=0.95
)
fig.show()

We can see, that this model performs great on the train data and misses it when the variance starts growing on the test data. Let's check out the residuals.

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=Y_test_extra,
    y=Y_test_extra-pipe_best_rf_extra_predictions,
    name='residual vs response',
    mode='markers',
    line=dict(color=px.colors.qualitative.Prism[4])
    )
)
fig.update_xaxes(title_text='actual number of drives in the test dataset')
fig.update_yaxes(title_text='residuals of the Random Forest model')
fig.update_layout(legend_x=0.1, legend_y=0.95, title = 'Residual plot')
fig.show()

The residuals demonstrate strong linear dependency on the target variable.

[Back to Content](#content)

## Conclusions <a id='conclusions'></a>

The raw dataset of Sweet Lift Taxi has ~26500 10-min observation points, no missing values, no duplicates. Initial upload showed, that:
1. The data exhibits a trend: the overall number of rides per day grew from 1300 in the beginning of March, 2018, up to 3700 rides per day by the end of August, 2018.
2. There is a seasonality component:
- The average number of rides diminishes on weekends and Tuesdays;
- There is a peak on Mondays;
- The numbers grow from Tuesday to Thursday.
3. The standard deviation grows with the growing number of rides; it exhibits weekly spikes as well - they start to be more expressed by the end of the period under investigation.

At the data exploration step we conducted tests for and showed that in the further modeling we should be aware of the following characteristics of the initial time series:
- heteroscedasticity;
- autocorrelative nature;
- overall nonlinear dependency on time.

Out initial feature engineering was focused on the following features:
- 24 hourly lag,
- day of week,  week, and hour,
- 4-hour slot rolling median (to make the model more robust).

For the resulting dataframe, we run a Random Forest model with hyperparameter search (GridSearchCV applied with custom cv function to account for the time series) and found a model with RMSE value of 22. We showed that the model residuals for the train dataset exhibit linear dependency on the target variable and suggested applying ARIMA-GARCH model.

We performed grid search for better ARIMA parameters, modeled residuals and run a GARCH model on the residuals. Based on the results of the ARIMA-GARCH modeling, ambiguous and time-consuming though, we suggested re-working the initial Random Forest by including additional features, namely rolling standard deviation (a proxy to variance) and a number of its lags.

While re-modeling, we added a rolling standard deviation term the list of features, including several lags of it. We re-run hyperparameter search and found a set of hyperparameters, which showed the same RMSE value of 22. On the test set, the model resulted in RMSE value of 44.

Further steps can include:
- further investigation of additioanl features,
- trying other models, besides RandomForest,
- further investigation of ARIMA-GARCH variations potential,
- developing a dynamic model.